In [3]:
import codecs
import os
import pandas as pd
import re
from os import listdir
from os.path import isfile, join

C:\Users\user\Anaconda3\lib\site-packages\pandas\compat\_optional.py:124: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
number_of_families = 13179
input_dir = 'gSpan_s150_l5'
file_names = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
context = pd.DataFrame()
for file_name in file_names:
    if 'members' not in file_name:
        continue
    graph_number = file_name.split('.')[0].split('_')[2]
    membership = pd.Series([0] * number_of_families)
    with codecs.open(join(input_dir, file_name), 'r', encoding='utf-8') as f:
        for line in f:
            membership[int(line.strip('\n').strip('\r').strip('F'))] = 1
    context[graph_number] = membership
context.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13179 entries, 0 to 13178
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0000    13179 non-null  int64
 1   0001    13179 non-null  int64
 2   0002    13179 non-null  int64
 3   0003    13179 non-null  int64
 4   0004    13179 non-null  int64
 5   0005    13179 non-null  int64
 6   0006    13179 non-null  int64
 7   0007    13179 non-null  int64
 8   0008    13179 non-null  int64
 9   0009    13179 non-null  int64
dtypes: int64(10)
memory usage: 1.0 MB


In [9]:
context['total_graph'] = context.sum(axis=1)
context.loc[context['total_graph'] > 0]

,0000,0001,0002,0003,0004,0005,0006,0007,0008,0009,total_graph
2,0,0,0,1,1,1,1,1,1,1,7
7,0,0,0,1,1,1,1,1,1,1,7
11,1,1,1,0,0,0,0,0,0,0,3
20,0,0,0,1,0,0,1,0,0,1,3
25,0,0,0,1,0,0,1,0,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...
7082,0,0,0,1,0,0,1,0,0,0,2
7084,0,0,0,1,0,0,1,0,0,0,2
7159,1,1,1,0,0,0,0,0,0,0,3
7232,0,0,0,1,0,0,1,0,0,1,3


In [10]:
context.drop(columns=['total_graph'], inplace=True)

In [3]:
context_file_name = 'context.csv'
context.to_csv(context_file_name, header=False, index=False)

In [12]:
os.system('java -jar AOCPosetBuilder.jar -i ' + context_file_name + ' -a HERMES -d posets/families_simplified.dot -f SIMPLIFIED -z')
os.system('dot -Tpdf posets/families_simplified.dot -o posets/families_simplified.pdf')
os.system('java -jar AOCPosetBuilder.jar -i ' + context_file_name + ' -a HERMES -d posets/families_full.dot -f FULL -z')

0

In [4]:
os.system('java -jar AOCPosetBuilder.jar -i ' + context_file_name + ' -a HERMES -d posets/families_minimal.dot -f MINIMAL')

0

Adding extent & intent size to the minimal version

In [8]:
file_min = codecs.open('posets/families_minimal.dot', 'w')
file_simp = 'posets/families_simplified.dot'
with codecs.open(file_simp) as f:
    for line in f:
        if 'graph' in line or 'rankdir' in line or '->' in line or line == '}':
            file_min.write(line)
            continue
        concept_id = re.search('<(.*)>', line).group(1)
        line_elements = line.split('|')
        line_elements[0] = line_elements[0].replace('> (', '> ' + concept_id + '|(').replace('I', 'Gr').replace('E', 'Fam')
        to_be_written = '|'.join(line_elements) + '\n'
        file_min.write(line_elements[0] + '}\"];\n')
file_min.close()
os.system('dot -Tpdf posets/families_minimal.dot -o posets/families_minimal.pdf')

0